In [2]:
import polars as pl
import pandas as pd
import re  
from time import time 
from collections import defaultdict

import spacy


In [3]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_text(text):
    if text is None:
        return ''
    tokens = tokenizer.tokenize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# from tqdm import tqdm
# tqdm.pandas()

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)

In [5]:
# # nlp = spacy.load("ru_core_news_sm",disable=['ner', 'parser'])
# def cleaning(doc):
#     txt = [token.lemma_ for token in doc if not token.is_stop]
#     return ' '.join(txt)

# brief_cleaning = [re.sub("[^A-Za-zА-Яа-я']+", ' ', str(row)).lower() for row in videos['video_title']]


In [6]:
# from time import time
# t = time()
# txt = [cleaning(doc) for doc in tqdm(nlp.pipe(brief_cleaning, batch_size=5000,n_process=14))]

# print(round((time() - t) / 60, 2))

In [7]:
!ls

 0_data_analysis.ipynb	       kozhin.ipynb   rutube.ipynb
 0_data_analysis_video.ipynb   __MACOSX       sinonim.ipynb
 alekseenkko.ipynb	       ranker.ipynb   txtAI.ipynb
 first_dataset_analyze.ipynb   rutube_case   'Untitled Folder'


In [8]:
!ls rutube_case/

'~$Описание файлов кейса от Rutube.xlsx'
 automarkup_10k.parquet
 automarkup.csv
 automarkup.parquet
 baseline-Copy1.ipynb
 baseline_doc_.ipynb
 baseline.ipynb
 baseline_to_new_faiss_engene.ipynb
 candidates.index
 catboost_info
 chunk
 data.pkl
 faiss_train_and_index.ipynb
 features
 features.parquet
 full_df.csv
 generated_candidates.parquet
 ind2videoid.json
 __init.__py
 kaggle.json
 manualmarkup.csv
 markov.ipynb
 pandarallel1
 paral
 qvant_1M.index
 ranker.ckpt
 requirements.txt
 sample_submission.csv
 search_engine.ipynb
 videos.parquet
'Описание файлов кейса от Rutube.xlsx'


In [9]:
import os 
os.chdir('rutube_case')

In [10]:
videos = pl.read_parquet('videos.parquet')

In [12]:
pl.Config(fmt_str_lengths=3000, tbl_rows=100)

In [13]:
videos.head(5)

video_id,video_title,channel_title,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime
str,str,str,"datetime[ns, UTC]",str,str,"datetime[ns, UTC]"
"""video_6484084""","""Palia. Рыбалка за час. От 3 до 12 000 голды!""","""Gamer Diary""",2022-04-13 10:23:19 UTC,"""UGC""","""Видеоигры""",2023-09-18 20:04:10 UTC
"""video_1522415""","""Прощание пройдет на чужбине...Нам будет не хватать Сенчуковой и Рыбина....""","""pntuser290""",2022-09-05 09:32:50 UTC,"""UGC""","""Разное""",2023-06-03 23:04:32 UTC
"""video_642363""","""#ДОБРОЛАП. ТВОРИМ ДОБРО ВМЕСТЕ!""","""Зоомагазин Четыре Лапы""",2022-04-13 08:07:35 UTC,"""UGC""","""Лайфстайл""",2022-04-15 14:18:15 UTC
"""video_10124356""","""8""","""МИЭТ""",2022-02-04 07:36:49 UTC,"""ПрофКонтент""","""Образование""",2017-09-16 11:15:08 UTC
"""video_16299346""","""Как обновить или переподключить аккаунт ВК в сервисе Best Liders""","""Руководство к Искусству""",2023-09-01 16:00:41 UTC,"""UGC""","""Разное""",2023-09-12 15:32:51 UTC


In [14]:
len(videos)

34404561

In [15]:
videos['v_channel_type'].value_counts()

v_channel_type,counts
str,u32
"""ТВ И СМИ""",461228
"""Originals""",2211
"""ПрофКонтент""",352462
"""Редакция""",59055
"""""",446
"""UGC""",33529159


In [16]:
videos['v_category'].value_counts().sort(by='counts',descending=True, nulls_last=True).head(10)

v_category,counts
str,u32
"""Разное""",25114889
"""Видеоигры""",1245935
"""Музыка""",825903
"""Лайфстайл""",701042
"""Образование""",509939
"""Развлечения""",444167
"""Спорт""",436328
"""Телепередачи""",378196
"""Люди и блоги""",360290


In [26]:
videos.groupby('v_channel_type','v_category').value_counts()

/tmp/ipykernel_27570/1796100598.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  videos.groupby('v_channel_type','v_category').value_counts()


AttributeError: 'GroupBy' object has no attribute 'value_counts'

In [17]:
manualmarkup = pl.read_csv('manualmarkup.csv',try_parse_dates=True)

In [18]:
manualmarkup.head(5)

query,video_id,query_date,sentiment
str,str,datetime[μs],str
"""эдисон""","""video_74232""",2023-11-03 18:00:00,"""useful"""
"""модная команда..модная я. leilazakirovna""","""video_45""",2023-11-03 18:00:00,"""uselessly"""
"""пираты карибского моря фильм""","""video_106361""",2023-11-03 18:00:00,"""could_be_better"""
"""data science""","""video_291105""",2023-11-03 18:00:00,"""useful"""
"""физикальное обследование ""","""video_57""",2023-11-03 18:00:00,"""uselessly"""


In [20]:
len(manualmarkup)

17240

In [19]:
manualmarkup['sentiment'].value_counts()

sentiment,counts
str,u32
"""сlickbait""",827
"""uselessly""",7819
null,4
"""exact""",404
"""useful""",4235
"""could_be_bette…",3951


In [20]:
len(manualmarkup.select('query').unique())

990

In [21]:
manualmarkup.group_by('query').agg(pl.col('video_id').count()).sort(by='video_id',descending=True, nulls_last=True)

query,video_id
str,u32
"""garrys mod""",32
"""hype realiti""",32
"""сердце клавы 1…",32
"""пацанки 7 сезо…",31
"""модная команда…",31
"""битва сильнейш…",30
"""лего""",30
"""скетч-шоу vell…",30
"""пушман""",30


In [22]:
manualmarkup.group_by('query').agg(pl.col('video_id').count()).sort(by='video_id',descending=True, nulls_last=True).filter(pl.col('video_id')>3)

query,video_id
str,u32
"""hype realiti""",32
"""garrys mod""",32
"""сердце клавы 1…",32
"""пацанки 7 сезо…",31
"""модная команда…",31
"""скетч-шоу vell…",30
"""битва сильнейш…",30
"""пушман""",30
"""лего""",30


# features

In [ ]:
features = pl.read_parquet('features.parquet')

In [55]:
len(features)

65452587

In [24]:
features.head()

video_id,report_date,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days
str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""video_1240127""","""2023-05-02""","""2017-02-28 01:…","""UGC""","""Путешествия""","""2017-02-28 01:…",0,0,0,0,0,0,0,283806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_3266704""","""2023-05-02""","""2022-03-26 20:…","""UGC""","""Видеоигры""","""2022-05-24 12:…",0,1,0,0,0,0,0,7242119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_26852455…","""2023-05-02""","""2022-03-04 15:…","""UGC""","""Техника и обор…","""2022-07-06 13:…",0,6,1,0,0,0,0,208516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_14041310…","""2023-05-02""","""2023-01-28 15:…","""UGC""","""Видеоигры""","""2023-02-02 10:…",0,6,0,0,0,0,0,282934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_6944943""","""2023-05-02""","""2019-06-18 14:…","""UGC""","""Спорт""","""2019-08-06 08:…",0,0,0,0,0,0,0,335040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# sample_submission

In [25]:
sub = pl.read_csv('sample_submission.csv')

In [26]:
sub.head(5)

query,video_id
str,str
"""query_0""","""video_0"""
"""query_0""","""video_1"""
"""query_0""","""video_2"""
"""query_0""","""video_3"""
"""query_0""","""video_4"""


# automarkup
Файл с кликами пользователя - каждая строка это пара query (запрос пользователя) и video_id (видео куда он кликнул по этому запросу), также представлены другие взаимодействия с видео

In [27]:
automarkup = pl.read_parquet('automarkup.parquet')

In [28]:
len(automarkup)

6234424

In [36]:
automarkup.select('datetime').min()

datetime
str
"""2023-05-03 00:…"


In [29]:
len(automarkup.select('video_id').unique())

412550

In [29]:
automarkup.head(5)

is_authorized,datetime,query,video_id,duration,position,watchtime,emotion,vtop,comment,channel,tv_show,season
bool,str,str,str,i64,f64,i64,i64,bool,i64,str,str,str
false,"""2023-07-03 14:…","""приют комедиан…","""video_3049590""",2279019,8.0,0,0,null,0,"""ТЕЛЕКАНАЛ ЗВЕЗ…","""Десять мгновен…","""Сезон"""
false,"""2023-07-20 04:…","""бабаян роман""","""video_3049590""",2279019,1.0,0,0,null,0,"""ТЕЛЕКАНАЛ ЗВЕЗ…","""Десять мгновен…","""Сезон"""
true,"""2023-08-10 17:…","""роман бабаян""","""video_3049590""",2279019,1.0,102,0,null,0,"""ТЕЛЕКАНАЛ ЗВЕЗ…","""Десять мгновен…","""Сезон"""
false,"""2023-07-03 17:…","""битва экстрасе…","""video_31592942…",4259447,45.0,3601,0,null,0,"""Экстрасенсы. Б…","""Экстрасенсы. Б…",null
false,"""2023-07-03 15:…","""битва сильнейш…","""video_31592942…",4259447,4.0,4054,0,null,0,"""Экстрасенсы. Б…","""Экстрасенсы. Б…",null


In [62]:
automarkup['emotion'].value_counts()

emotion,counts
i64,u32
8,44
4,566
12,38
5,418
7,287
6,1981
3,991
2,3618
10,14


In [63]:
automarkup['season'].value_counts()

season,counts
str,u32
"""ШЕРЛОК ХОЛМС (…",179
"""ДОМ 2 Реальная…",9
"""Бородина проти…",6
"""Сезон 11""",1219
"""Сезон 21""",145
"""Сезон 2""",7158
"""ДОМ-2: Город л…",1
"""Сезон 5""",13575
"""0 сезон""",4


In [66]:
automarkup['is_authorized'].value_counts()

is_authorized,counts
bool,u32
true,330640
false,5903784


In [67]:
automarkup['tv_show'].value_counts()

tv_show,counts
str,u32
"""Хроники москов…",36
"""Танцы без прав…",10
"""Московская пле…",2
"""1937. Год стра…",1
"""Тропа смерти /…",4
"""Далекие странн…",39
"""SHAPKA""",12
"""Киноистория. В…",2
"""Ди.Грэй-мен / …",101


In [31]:
automarkup.describe()

describe,is_authorized,datetime,query,video_id,duration,position,watchtime,emotion,vtop,comment,channel,tv_show,season
str,f64,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str
"""count""",6.234424e6,"""6234424""","""6234424""","""6234424""",6.234424e6,6.234424e6,6.234424e6,6.234424e6,6.234424e6,6.234424e6,"""6234424""","""6234424""","""6234424"""
"""null_count""",0.0,"""0""","""0""","""0""",0.0,0.0,0.0,0.0,6.21255e6,0.0,"""213""","""2702589""","""5543943"""
"""mean""",0.053035,null,null,null,3.1858e6,3.537818,1110.193369,0.0065,1.0,0.004891,null,null,null
"""std""",0.224102,null,null,null,3.0241e6,7.102413,1847.504495,0.158952,0.0,0.069762,null,null,null
"""min""",0.0,"""2023-05-03 00:…",""" ""","""video_10000005…",0.0,1.0,-1.0,0.0,1.0,0.0,""" CAUCASIAN UNI…",""" Пчёлка Майя и…","""0 сезон"""
"""25%""",null,null,null,null,378763.0,1.0,11.0,0.0,null,0.0,null,null,null
"""50%""",null,null,null,null,2.557534e6,1.0,151.0,0.0,null,0.0,null,null,null
"""75%""",null,null,null,null,5.597534e6,3.0,1591.0,0.0,null,0.0,null,null,null
"""max""",1.0,"""2023-11-03 23:…","""🧙‍♀️ееееее""","""video_9999973""",2.1475e9,100.0,1.171925e6,12.0,1.0,1.0,"""Ｒｅｂｏｒｎ""","""№13""","""сезон"""


In [68]:
!ls features

features_aug.parquet  features_may.parquet  features_sep.parquet
features_jul.parquet  features_nov.parquet
features_jun.parquet  features_oct.parquet


In [69]:
features_aug = pl.read_parquet('features/features_aug.parquet')

In [70]:
features_aug

video_id,report_date,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days
str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""video_2947178""","""2023-08-02""","""2015-02-11 11:…","""UGC""","""Музыка""","""2015-04-27 16:…",0,4,1,0,0,0,0,218440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_33435616…","""2023-08-02""","""2022-04-09 18:…","""UGC""","""Музыка""","""2022-04-26 17:…",0,17,0,0,0,0,0,226349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_17123907…","""2023-08-02""","""2014-07-27 10:…","""UGC""","""Разное""","""2014-11-15 20:…",0,2,0,0,0,0,0,15208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_19099582…","""2023-08-02""","""2022-03-05 05:…","""UGC""","""Кулинария""","""2022-03-07 13:…",0,0,0,0,0,0,0,333834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_20408586…","""2023-08-02""","""2022-09-27 16:…","""UGC""","""Аниме""","""2022-11-18 11:…",0,14,0,0,0,0,0,267222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_5090847""","""2023-08-02""","""2022-03-13 12:…","""UGC""","""Видеоигры""","""2023-04-24 17:…",0,4,1,0,0,0,0,1368517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_17896697…","""2023-08-02""","""2022-03-15 06:…","""ТВ И СМИ""","""Телепередачи""","""2022-04-13 12:…",0,41,6,0,0,0,0,2644010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0
"""video_10477427…","""2023-08-02""","""2021-02-28 17:…","""UGC""","""Развлечения""","""2022-03-21 19:…",0,6,1,0,0,0,0,1322632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""video_27183655…","""2023-08-02""","""2023-01-30 07:…","""UGC""","""Лайфстайл""","""2023-03-06 15:…",0,5,0,0,0,0,0,1097910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
